### Create a restaurant recommendation system based on user preferences.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform
pd.reset_option('display.max_rows')

In [4]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv("restaurant data.csv")
print(df.head())

   Restaurant ID         Restaurant Name  Country Code              City  \
0        6317637        Le Petit Souffle           162       Makati City   
1        6304287        Izakaya Kikufuji           162       Makati City   
2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
3        6318506                    Ooma           162  Mandaluyong City   
4        6314302             Sambo Kojin           162  Mandaluyong City   

                                             Address  \
0  Third Floor, Century City Mall, Kalayaan Avenu...   
1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   

                                     Locality  \
0   Century City Mall, Poblacion, Makati City   
1  Little Tokyo, Legaspi Village, Makati City   
2  Edsa Shangri-La, Ortigas, Mandaluyong City   
3      SM 

In [8]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [10]:
dfRS = df[['Restaurant ID','Restaurant Name','Cuisines','Aggregate rating','Votes']]
dfRS

,Restaurant ID,Restaurant Name,Cuisines,Aggregate rating,Votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml? Gurme,Turkish,4.1,788
9547,5908749,Ceviz A??ac?,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A???k Kahve,Restaurant Cafe,4.0,901


In [12]:
def dataDesc():
    listItem = []
    for col in dfRS.columns :
        listItem.append(
            [col,
            dfRS[col].dtype,
            dfRS[col].isna().sum(),
            round(dfRS[col].isna().sum()/len(dfRS)*100,2),
            dfRS[col].nunique(),
            list(dfRS[col].drop_duplicates().sample(2).values)]
        )
    descData = pd.DataFrame(data = listItem,
                            columns = ['Column','Data Type', 'Missing Value',
                                        'Pct Missing Value', 'Num Unique', 'Unique Sample'])
    return descData

dataDesc()

,Column,Data Type,Missing Value,Pct Missing Value,Num Unique,Unique Sample
0,Restaurant ID,int64,0,0.00,9551,"[18287390, 18138421]"
1,Restaurant Name,object,0,0.00,7446,"[GPW - The Food Hub, Yumbo Bites]"
2,Cuisines,object,9,0.09,1825,"[Chinese, Seafood, Cafe, American, Continental..."
3,Aggregate rating,float64,0,0.00,33,"[2.8, 2.2]"
4,Votes,int64,0,0.00,1012,"[1317, 1065]"


In [14]:
dfRS = dfRS.dropna()

In [16]:
dfRS

,Restaurant ID,Restaurant Name,Cuisines,Aggregate rating,Votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml? Gurme,Turkish,4.1,788
9547,5908749,Ceviz A??ac?,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A???k Kahve,Restaurant Cafe,4.0,901


In [18]:
dfRS = dfRS.rename(columns={'Restaurant ID': 'restaurant_id'})
dfRS = dfRS.rename(columns={'Restaurant Name': 'restaurant_name'})
dfRS = dfRS.rename(columns={'Cuisines': 'cuisines'})
dfRS = dfRS.rename(columns={'Aggregate rating': 'aggregate_rating'})
dfRS = dfRS.rename(columns={'Votes': 'votes'})

In [20]:
dfRS

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
0,6317637,Le Petit Souffle,"French, Japanese, Desserts",4.8,314
1,6304287,Izakaya Kikufuji,Japanese,4.5,591
2,6300002,Heat - Edsa Shangri-La,"Seafood, Asian, Filipino, Indian",4.4,270
3,6318506,Ooma,"Japanese, Sushi",4.9,365
4,6314302,Sambo Kojin,"Japanese, Korean",4.8,229
...,...,...,...,...,...
9546,5915730,Naml? Gurme,Turkish,4.1,788
9547,5908749,Ceviz A??ac?,"World Cuisine, Patisserie, Cafe",4.2,1034
9548,5915807,Huqqa,"Italian, World Cuisine",3.7,661
9549,5916112,A???k Kahve,Restaurant Cafe,4.0,901


In [22]:
dfRS.duplicated().sum()

0

In [24]:
dfRS['restaurant_name'].duplicated().sum()

2105

In [26]:
dfRS['restaurant_name'].value_counts()

restaurant_name
Cafe Coffee Day             83
Domino's Pizza              79
Subway                      63
Green Chick Chop            51
McDonald's                  48
                            ..
The Town House Cafe          1
The G.T. Road                1
The Darzi Bar & Kitchen      1
Smoke On Water               1
Walter's Coffee Roastery     1
Name: count, Length: 7437, dtype: int64

In [28]:
dfRS = dfRS.sort_values(by=['restaurant_name','aggregate_rating'],ascending=False)

In [30]:
dfRS[dfRS['restaurant_name']=="Domino's Pizza"].head()

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
3031,143,Domino's Pizza,"Pizza, Fast Food",3.7,336
1844,5065,Domino's Pizza,"Pizza, Fast Food",3.6,146
2448,15078,Domino's Pizza,"Pizza, Fast Food",3.6,86
7618,18263236,Domino's Pizza,"Pizza, Fast Food",3.6,24
8437,384,Domino's Pizza,"Pizza, Fast Food",3.6,547


In [32]:
dfRS = dfRS.drop_duplicates('restaurant_name',keep='first')
dfRS

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
3120,18222559,{Niche} - Cafe & Bar,"North Indian, Chinese, Italian, Continental",4.1,492
9334,7100938,wagamama,"Japanese, Asian",3.7,131
9454,6401789,tashas,"Cafe, Mediterranean",4.1,374
4659,18361747,t Lounge by Dilmah,"Cafe, Tea, Desserts",3.6,34
9385,6113857,sketch Gallery,"British, Contemporary",4.5,148
...,...,...,...,...,...
8692,18317511,#Urban Caf??,"North Indian, Chinese, Italian",3.3,49
6998,18336489,#OFF Campus,"Cafe, Continental, Italian, Fast Food",3.7,216
2613,18311951,#InstaFreeze,Ice Cream,0.0,2
9148,18378803,#Dilliwaala6,North Indian,3.7,124


In [34]:
dfRS['restaurant_name'].value_counts()

restaurant_name
{Niche} - Cafe & Bar        1
French Omelette             1
Four Queens Dairy Cream     1
Fourteen Eleven Tea Cafe    1
Fozzie's Pizzaiolo          1
                           ..
Pizza Man                   1
Pizza Point                 1
Pizza Station               1
Pizza Street                1
#45                         1
Name: count, Length: 7437, dtype: int64

In [36]:
dfRS = dfRS[dfRS['aggregate_rating']>=4.0]
dfRS

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
3120,18222559,{Niche} - Cafe & Bar,"North Indian, Chinese, Italian, Continental",4.1,492
9454,6401789,tashas,"Cafe, Mediterranean",4.1,374
9385,6113857,sketch Gallery,"British, Contemporary",4.5,148
1837,18418247,feel ALIVE,"North Indian, American, Asian, Biryani",4.7,69
440,17616266,Zunzi's,"International, Mediterranean, Sandwich",4.5,796
...,...,...,...,...,...
1468,18408054,19 Flavours Biryani,"Mughlai, Hyderabadi",4.1,84
2484,18233317,145 Kala Ghoda,"Fast Food, Beverages, Desserts",4.2,1606
2292,2100784,11th Avenue Cafe Bistro,"Cafe, American, Italian, Continental",4.1,377
751,2600031,10 Downing Street,"North Indian, Chinese",4.0,257


In [38]:
dfRS['cuisines'] = dfRS['cuisines'].str.split(', ')
dfRS

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
3120,18222559,{Niche} - Cafe & Bar,"[North Indian, Chinese, Italian, Continental]",4.1,492
9454,6401789,tashas,"[Cafe, Mediterranean]",4.1,374
9385,6113857,sketch Gallery,"[British, Contemporary]",4.5,148
1837,18418247,feel ALIVE,"[North Indian, American, Asian, Biryani]",4.7,69
440,17616266,Zunzi's,"[International, Mediterranean, Sandwich]",4.5,796
...,...,...,...,...,...
1468,18408054,19 Flavours Biryani,"[Mughlai, Hyderabadi]",4.1,84
2484,18233317,145 Kala Ghoda,"[Fast Food, Beverages, Desserts]",4.2,1606
2292,2100784,11th Avenue Cafe Bistro,"[Cafe, American, Italian, Continental]",4.1,377
751,2600031,10 Downing Street,"[North Indian, Chinese]",4.0,257


In [40]:
dfRS = dfRS.explode('cuisines')
dfRS

,restaurant_id,restaurant_name,cuisines,aggregate_rating,votes
3120,18222559,{Niche} - Cafe & Bar,North Indian,4.1,492
3120,18222559,{Niche} - Cafe & Bar,Chinese,4.1,492
3120,18222559,{Niche} - Cafe & Bar,Italian,4.1,492
3120,18222559,{Niche} - Cafe & Bar,Continental,4.1,492
9454,6401789,tashas,Cafe,4.1,374
...,...,...,...,...,...
2292,2100784,11th Avenue Cafe Bistro,Italian,4.1,377
2292,2100784,11th Avenue Cafe Bistro,Continental,4.1,377
751,2600031,10 Downing Street,North Indian,4.0,257
751,2600031,10 Downing Street,Chinese,4.0,257


In [42]:
dfRS['cuisines'].value_counts()

cuisines
North Indian    270
Italian         237
Chinese         200
Continental     199
Cafe            177
               ... 
Durban            1
Iranian           1
Awadhi            1
Persian           1
Irish             1
Name: count, Length: 128, dtype: int64

In [44]:
xTabRestoCuisines = pd.crosstab(dfRS['restaurant_name'],dfRS['cuisines'])

In [46]:
xTabRestoCuisines

cuisines,Afghani,African,American,Andhra,Arabian,Argentine,Asian,Asian Fusion,Australian,Awadhi,...,Teriyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegetarian,Vietnamese,Western,World Cuisine
restaurant_name,,,,,,,,,,,,,,,,,,,,,
'Ohana,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10 Downing Street,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11th Avenue Cafe Bistro,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
145 Kala Ghoda,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19 Flavours Biryani,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zunzi's,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
feel ALIVE,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sketch Gallery,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
xTabRestoCuisines.loc['feel ALIVE'].values

array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [50]:
dfRS['restaurant_name'].sample(20, random_state=101)

509     Mr. Dunderbak's Biergarten and Marketplatz
17                                      Balay Dako
327                                        Rookery
4639                                     Big Chill
9333                                         Olive
4321                               Tourist Janpath
1838                                      Matchbox
9259                      Mekong - Hotel GreenPark
9359                              Mimi's Bakehouse
9472                    Capital Craft Beer Academy
3107                                  Odeon Social
705                                   Kanha Sweets
5207                                    Vero Gusto
9441                                  Truth Coffee
4088      Tian - Asian Cuisine Studio - ITC Maurya
9343                                        Bodega
9374                             Ting Thai Caravan
3107                                  Odeon Social
9513                                    The Sizzle
2328                         Sq

In [52]:
print(jaccard_score(xTabRestoCuisines.loc["Olive Bistro"].values,
                    xTabRestoCuisines.loc["Rose Cafe"].values))

0.3333333333333333


In [54]:
jaccardDist = pdist(xTabRestoCuisines.values, metric='jaccard')
jaccardMatrix = squareform(jaccardDist)
jaccardSim = 1 - jaccardMatrix
dfJaccard = pd.DataFrame(
    jaccardSim,
    index=xTabRestoCuisines.index,
    columns=xTabRestoCuisines.index)

dfJaccard

restaurant_name,'Ohana,10 Downing Street,11th Avenue Cafe Bistro,145 Kala Ghoda,19 Flavours Biryani,1918 Bistro & Grill,2 Dog,22nd Parallel,3 Wise Monkeys,38 Barracks,...,Zoe,Zoeys Pizzeria,Zolocrust - Hotel Clarks Amer,Zombie Burger + Drink Lab,Zuka Choco-la,Zunzi's,feel ALIVE,sketch Gallery,tashas,{Niche} - Cafe & Bar
restaurant_name,,,,,,,,,,,,,,,,,,,,,
'Ohana,1.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.00,0.000000
10 Downing Street,0.0,1.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.200000,...,0.0,0.00,0.0,0.0,0.000000,0.00,0.200000,0.0,0.00,0.500000
11th Avenue Cafe Bistro,0.0,0.0,1.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.333333,...,0.0,0.00,0.4,0.0,0.000000,0.00,0.142857,0.0,0.20,0.333333
145 Kala Ghoda,0.0,0.0,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.2,0.00,0.0,0.2,0.333333,0.00,0.000000,0.0,0.00,0.000000
19 Flavours Biryani,0.0,0.0,0.000000,0.0,1.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.0,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zunzi's,0.0,0.0,0.000000,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000,...,0.0,0.25,0.0,0.0,0.000000,1.00,0.000000,0.0,0.25,0.000000
feel ALIVE,0.0,0.2,0.142857,0.0,0.0,0.0,0.166667,0.0,0.0,0.600000,...,0.0,0.00,0.0,0.0,0.000000,0.00,1.000000,0.0,0.00,0.142857
sketch Gallery,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,1.0,0.00,0.000000


In [56]:
dfRS['restaurant_name'].sample(20)

9517                              Timboo Cafe
156     Guido's Original New York Style Pizza
3703               Sakley's The Mountain Cafe
784                                Food Fever
603                                Grub Shack
429                             Marukame Udon
4673                     SodaBottleOpenerWala
643                                Thaaliwala
637                           Sheroes Hangout
7045                            Wood Box Cafe
210               Christian and Jake's Bistro
2309                          Dock Forty Five
9379                                Flat Iron
2396                             French Toast
1441                      The Hangout by 1861
112                                Five & Ten
4553                                  Classic
2438                         Cappuccino Blast
1837                               feel ALIVE
3104                                  HotMess
Name: restaurant_name, dtype: object

### Final Recommendation System

In [59]:
resto = 'Ooma'

sim = dfJaccard.loc[resto].sort_values(ascending=False)

sim = pd.DataFrame({'restaurant_name': sim.index, 'simScore': sim.values})
sim = sim[(sim['restaurant_name']!= resto) & (sim['simScore']>=0.7)].head(5)

# Merge The Rating
RestoRec = pd.merge(sim,dfRS[['restaurant_name','aggregate_rating']],how='inner',on='restaurant_name')
FinalRestoRec = RestoRec.sort_values('aggregate_rating',ascending=False).drop_duplicates('restaurant_name',keep='first')

In [61]:
FinalRestoRec

,restaurant_name,simScore,aggregate_rating
6,Sushi Masa,1.0,4.9
2,Roka,1.0,4.6
4,Nobu,1.0,4.4
0,Nagai,1.0,4.3
8,Osaka,1.0,4.2


### Conclusion

In [ ]:
The above Data will show up to top 5 recommended restaurants with the best rating, the rating is also curated only 4 and above, 
so the reccomendation system porvide good rating objectively.